# T-Maze Aggregate Simulations with Generalized FE

In [ ]:
using Pkg
Pkg.activate(".")
# Pkg.instantiate()

In [ ]:
using RxInfer, LinearAlgebra, ProgressMeter, Plots

## Agent

In [ ]:
include("goal_observation.jl")

# Define the generative model
@model function t_maze(A_s, D_s, x)
    u = datavar(Matrix{Int64}, 2) # Policy for evaluations
    z = randomvar(2) # Latent states
    c = datavar(Vector{Float64}, 2) # Goal prior statistics

    z_0 ~ Categorical(D_s) # State prior
    A ~ MatrixDirichlet(A_s) # Observation matrix prior

    z_k_min = z_0
    for k=1:2
        z[k] ~ Transition(z_k_min, u[k])
        c[k] ~ GoalObservation(z[k], A) where {
            meta=GeneralizedMeta(x[k]), 
            pipeline=GeneralizedPipeline(vague(Categorical,8))} # With breaker message

        z_k_min = z[k] # Reset for next slice
    end
end

# Define constraints on the variational density
@constraints function structured(approximate::Bool)
    q(z_0, z, A) = q(z_0, z)q(A)
    if approximate # Sampling approximation on A required for t<3
        q(A) :: SampleList(20)
    end
end

## Simulation

In [ ]:
# Define experimental setting
α = 0.9; c = 2.0 # Reward probability and utility
R = 100 # Number of runs
S = 30 # Number of trials
seed = 666 # Randomizer seed

include("helpers.jl")
include("visualizations.jl")
include("environment.jl")
include("agent.jl")

(A, B, C, D) = constructABCD(α, c)
(A_0, D_0) = constructPriors() # Construct prior statistics for A and D

wins = [Vector{Float64}(undef, S) for ri=1:R]
params = Vector{Matrix}(undef, R)

@showprogress for r=1:R
    rs = generateGoalSequence(S) # Returns random goal sequence
    (reset, execute, observe) = initializeWorld(A, B, C, D, rs) # Let there be a world
    (infer, act) = initializeAgent(A_0, B, C, D_0) # Let there be a constrained agent

    # Step through the experimental protocol
    As = Vector{Matrix}(undef, S) # Posterior statistics for A
    Gs = [Vector{Matrix}(undef, 3) for s=1:S] # Free energy values per time
    as = [Vector{Int64}(undef, 2) for s=1:S] # Actions per time
    os = [Vector{Vector}(undef, 2) for s=1:S] # Observations (one-hot) per time
    for s = 1:S
        reset(s) # Reset world
        for t=1:2
            (Gs[s][t], _) = infer(t, as[s], os[s])
                as[s][t] = act(t, Gs[s][t])
                            execute(as[s][t])
                os[s][t] = observe()
        end
        (Gs[s][3], As[s]) = infer(3, as[s], os[s]) # Learn at t=3
    end
    wins[r] = extractWins(os)
    params[r] = deepcopy(As[end])
end
;

In [ ]:
using FileIO, JLD2
FileIO.save("figures/wins_$(R)_$(S).jld2","wins",wins,"params",params,"R",R,"S",S)

## Results

In [ ]:
# Load data from file
using FileIO, JLD2, Plots, Statistics, Distributions

pairs = FileIO.load("figures/wins_100_30.jld2")
wins = pairs["wins"]
params = pairs["params"]
R = pairs["R"]
S = pairs["S"]
;

In [ ]:
win_counts = Int64.(sum.(wins))
f_ideal = pdf.(Binomial(S, 0.9), 1:S).*R

# Plot histogram for number of wins per run
histogram(win_counts, bins=-1:1:S, 
          dpi=300,
          size=(400,250),
          color=:gray, 
          label=false, 
          xlabel="Wins per Run", 
          ylabel="Run Count",
          legend=120,
          xticks=((0:10:30).-0.5, 0:10:30))

plot!((1:S).-0.5, f_ideal, linestyle=:dash, lw=2, color=:black, label=false)

savefig("figures/GFE_hist.png")

In [ ]:
m = mean(wins)

# Plot average wins per trial
plot(1:S, m, ylim=(0,1),
     dpi=300,
     size=(400,250),
     color=:black, 
     lw=2, 
     label=false, 
     xlabel="Simulation Trial (s)", 
     ylabel="Win Average")
plot!(1:S, 0.9.*ones(S), color=:black, linestyle=:dash, lw=2, label=false, legend=0)

savefig("figures/GFE_wins.png")